# 0. Band gap regression using Decision Trees

In this notebook, we will use decision trees to solve regression problems. 

The dataset used here originates from a project to build a surrogate model for predicting the band gap of a material from its composition. This surrogate model was used to replace expensive qunatum mecahnical calculations in virtual high-throughput screening of materials for application as photocatalysts. The paper was published in [Chemistry of Materials](https://pubs.acs.org/doi/abs/10.1021/acs.chemmater.9b01519). 

The aim of this work is to use regression trees and learn how to tune hyperparameters for best performance.

In [ ]:
# sklearn
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import sklearn.datasets

# helpers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os.path import join

# need some certainty in data processing
np.random.seed(1234)


## Google Cloud Storage Boilerplate

The following two cells have some boilerplate to mount the Google Cloud Storage bucket containing the data used for this notebook to your Google Colab file system. **Even you are not using Google Colab, please make sure you run these two cells.** 

To access the data from Google Colab, you need to:

1. Run the first cell;
2. Follow the link when prompted (you may be asked to log in with your Google account);
3. Copy the Google SDK token back into the prompt and press `Enter`;
4. Run the second cell and wait until the data folder appears.

If everything works correctly, a new folder called `sciml-workshop-data` should appear in the file browser on the left. Depending on the network speed, this may take one or two minutes. Ignore the warning "You do not appear to have access to project ...". If you are running the notebook locally or you have already connected to the bucket, these cells will have no side effects.

In [ ]:
# variables passed to bash; do not change
project_id = 'sciml-workshop'
bucket_name = 'sciml-workshop'
colab_data_path = '/content/sciml-workshop-data/'

try:
    from google.colab import auth
    auth.authenticate_user()
    google_colab_env = 'true'
    data_path = colab_data_path
except:
    google_colab_env = 'false'
    ###################################################
    ######## specify your local data path here ########
    ###################################################
    with open('../local_data_path.txt', 'r') as f: data_path = f.read().splitlines()[0]

In [ ]:
%%bash -s {google_colab_env} {colab_data_path} {project_id} {bucket_name}

# running locally
if ! $1; then
    echo "Running notebook locally."
    exit
fi

# already mounted
if [ -d $2 ]; then
    echo "Data already mounted."
    exit
fi

# mount the bucket
echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
apt -qq update
apt -qq install gcsfuse
gcloud config set project $3
mkdir $2
gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 $4 $2

---

# 1. Load the dataset

Our data are stored in the pickle file `Material/oxides.pickle`. We load this file into a `pandas.DataFrame` object, an efficient interface to manage column-wise, heterogeneous tabular data. 

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
oxides = pd.read_pickle(join(data_path, 'Material/oxides.pickle'))
```
    
</p>
</details>

We can check all the columns presented in the dataframe:

To read data from one of the columns, use the `values` attribute, for example:

### Description of the dataset

In this practical we are attempting to learn a model that can predict the band gap (energy separation between occupied and un-occupied orbitals) of a material. So we need to set this value as the property to be predicted $y$ This data is stored in the dataframe column called `gllbsc_gap` and we set this to be y by running the cell below:

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
# read a single column
y = oxides['gllbsc_gap'].values
```
    
</p>
</details>

We can then use the other properties in the dataset, or a combination of them as *features* ($X$) for our model. For example we could set X to be defined by two features by running the cell below:

In [ ]:
# read multiple columns and combine them to a matrix
X = oxides[['MagpieData minimum Number', 'MagpieData maximum Number']].values
print(X.shape)

# 2. Build a baseline model

In regression, we attempt to fit a model, $y = f(x)$, where $x$ and $y$ are multi-dimensional data of rank $M$ and $N$, respectively, and $f: \mathbb{R}^M\rightarrow\mathbb{R}^N$ our regression model. In this notebook, $y$ will always be `gllbsc_gap` (so $N=1$), which represents the band gap, and $x$ a combination of the descriptors (all the other columns), each giving the measurement of a certain physical property. 

---

## Linear regression: a starter

Linear regression is the simplest regression algorithm in machine learning. Many people do not even regard it as a machine learning algorithm because it is explicitly programmed. Still, it serves as a good start to learn some basic concepts.


## Univariate regression

In univariate linear regression we have the equation:
$y = mx + c$
and we are attempting to find the best values for $m$ and $c$

In a univariate regression, the input rank $M=1$. For instance, let us try `MagpieData avg_dev Electronegativity` as $x$:

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
# read X
X = oxides['MagpieData avg_dev Electronegativity'].values
# we need to append a dummy dimension to X for univariate regression
# to keep the input dimensions consistent with multivariate regression
X = X.reshape(-1, 1)
# read y
y = oxides['gllbsc_gap'].values
```
    
</p>
</details>

Now we can use linear regression to fit the data and make predictions:

In [ ]:
# fit linear regression model
model = LinearRegression().fit(X, y)
# make predictions
y_pred = model.predict(X)

When we have fitted the model we now want to use some *metrics* to *evaluate* the model performance. Remember the mean squared error and mean absolute error from your lectures. We will now calculate them for the model:

In [ ]:
# compute some fitting error
print('MSE = %f eV' % metrics.mean_squared_error(y, y_pred))
print('MAE = %f eV' % metrics.mean_absolute_error(y, y_pred))

We can also plot the predicted versus the real values to get a visual feel for how well the fitting worked.

In [ ]:
plt.figure(dpi=100)
plt.scatter(y, y_pred)
plt.xlabel('Eg True (eV)')
plt.ylabel('Eg Predicted (eV)')
plt.show()

## Exercise 

By changeing the feature in the $X$ values above try a number of different features. How does it affect the quality of fitting? Report the feature and the MAE and MSE scores in the table below. *Note* to edit the contets of this cell, simply double click on the cell.

| Feature | MAE (eV) | MSE (eV) |
|---------|----------|----------|
|         |          |          |
|         |          |          |
|         |          |          |
|         |          |          |

## Multivariate regression

In a multivariate regression, the input rank $M>1$. Therefore, we will choose a few descriptor to form $x$. Here we choose three descriptors ($M=3$):

In [ ]:
# read X
X = oxides[['MagpieData avg_dev CovalentRadius', 
            'MagpieData avg_dev Electronegativity', 
            'MagpieData maximum NsValence']].values

And the rest is the same as univariate regression:

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
# fit linear regression model
model = LinearRegression().fit(X, y)

# make predictions
y_pred = model.predict(X)

# compute some fitting error
print('MSE = %f' % metrics.mean_squared_error(y, y_pred))
print('MAE = %f' % metrics.mean_absolute_error(y, y_pred))

# plot the original and predicted data against each other
plt.figure(dpi=100)
plt.scatter(y, y_pred)
plt.xlabel('Eg True (eV)')
plt.ylabel('Eg Predicted (eV)')
plt.show()
```
    
</p>
</details>

## Exercise 

By changeing the features in the $X$ values above try a number of different feature combinations. How does it affect the quality of fitting? Report the feature and the MAE and MSE scores in the table below. *Note* to edit the contets of this cell, simply double click on the cell.

| Feature | MAE (eV) | MSE (eV) |
|---------|----------|----------|
|         |          |          |
|         |          |          |
|         |          |          |
|         |          |          |

---

# Gradient Boosting Regression

Gradient boosting is a method for building an ensemble of weak learners to constitute a single strong learner. We build a series of decision trees, each subsequent tree taking in information about the residuals (errors) from the previous trees. In principle, the fitting should improve each time a new tree is added. 

## 1. Create the regressor

In `sklearn`, a gradient boosting regressor is created by

```python
GradientBoostingRegressor(loss=<str>, max_depth=<int>, learning_rate=<float>,
                          min_samples_split=<int>, min_samples_leaf=<int>, 
                          max_features=<int>, subsample=<float>, n_estimators=<int>)
```


The hyperparameters we need to set include:

* `loss`: a loss function to be minimised. We will use 'lad', which is basically MAE.
* `max_depth`: the maximum depth limits the number of nodes in the trees; its best value depends on the interaction of the input variables; we will start with 10 and can tune it later.
* `learning_rate`: learning rate shrinks the contribution of each tree; there is a trade-off between learning rate and boosting steps; we will start with 0.015 and can tune it later.
* `min_samples_split`: the minimum number of samples required to split an internal node; we will start with 50 and can tune it later.
* `min_samples_leaf`: the minimum number of samples required to be at a leaf node; we set this as 1.
* `max_features`: the number of features to consider when looking for the best split; we will use the number of features in the data.
* `subsample`: the fraction of samples to be used for fitting the individual trees; if smaller than 1.0, this results in Stochastic Gradient Boosting. we will start with 0.9 and can tune it later.
* `n_estimators`: the number of boosting steps or decision trees; we will start with 300 and can tune it later.

**NOTE**: Simply adding more trees can lead to overfitting. Gradient boosting is quite robust against overfitting, but we will have to look out for this.

In [ ]:
# create the regressor
gbr = GradientBoostingRegressor(loss='lad', max_depth=10, learning_rate=0.015,
                                min_samples_split=50, min_samples_leaf=1, 
                                max_features=len(oxides.columns)-1, subsample=0.9, 
                                n_estimators=300)

## 2. Fit the regressor

Here we combine all the descriptors to form $x$ and fit the model:

In [ ]:
# combine all the columns into X
cols = [a for a in list(oxides.columns) if a not in ['gllbsc_gap']]
X = oxides[cols].values
print('Shape of X: %s' % str(X.shape))

# fit the model
gbr.fit(X, y)

After fitting the model, we can make predictions and plot them against the original data. The fit has shown a significant improvement over linear regression. 

In [ ]:
# make predictions
y_pred = gbr.predict(X)

# plot the original and predicted data against each other
plt.figure(dpi=100)
plt.scatter(y, y_pred)
plt.plot([y.min(), y.max()], [y_pred.min(), y_pred.max()],'r')
plt.xlabel('Eg True (eV)')
plt.ylabel('Eg Predicted (eV)')
plt.show()

## 3. Cross validation

Cross-validation (CV) allows us to evaluate the out-of-sample goodness-of-fit of the regressor without sparing a validation set. In the basic approach, as called the k-fold CV, the training set is split into $k$ subsets, each serving as the validation set to evaluate the model trained with the other $k-1$ subsets. This approach can be computationally expensive but does not waste too much data (as is the case when fixing an arbitrary validation set), which is a major advantage for problems with limited data. Note that a lower CV score means better goodness of fit.

In the following cell, we compute the scores using 5 folds (so 20% of data for each validation) and the negative MAE as the metric:

In [ ]:
# compute cross validation score
scores = cross_val_score(gbr, X, y, cv=5, scoring='neg_mean_absolute_error')
print('Cross validation score: {}'.format(-1 * np.mean(scores)))

## 4. Boosting rate and overfitting

Let us split the dataset 80:20 into training and test sets. Re-fit the model using the training set only. We can then use some built-in methods of `GradientBoostingRegressor` to get training and test scores at each iteration of boosting. This way, we can check if we have insufficient boosting layers or perhaps we have too many and thus suffer overfitting.


In [ ]:
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# fit with training set
gbr.fit(X_train, y_train)

# compute test score at each boosting step
test_score = np.zeros((300,), dtype=np.float64)
for i, y_pred in enumerate(gbr.staged_predict(X_test)):
    test_score[i] = gbr.loss_(y_test, y_pred)

# plot the scores
plt.figure(dpi=100)
plt.plot(gbr.train_score_, label='Loss on training set')
plt.plot(test_score, label='Loss on test set')
plt.legend()
plt.show()

Notice that the loss of both training and test are still decreasing at 300 steps. We can try to increase the boosting steps to 500 and see if we can still get improvements. If the test score stops increasing, we are probably in a good place to stop extending the model. 

In [ ]:
# create the regressor with more boosting steps
gbr500 = GradientBoostingRegressor(loss='lad', max_depth=10, learning_rate=0.015,
                                   min_samples_split=50, min_samples_leaf=1, 
                                   max_features=len(oxides.columns)-1, subsample=0.9, 
                                   n_estimators=500)

# fit with training set
gbr500.fit(X_train, y_train)

# compute test score at each boosting step
test_score = np.zeros((500,), dtype=np.float64)
for i, y_pred in enumerate(gbr500.staged_predict(X_test)):
    test_score[i] = gbr500.loss_(y_test, y_pred)

# plot the scores
plt.figure(dpi=100)
plt.plot(gbr500.train_score_, label='Loss on training set')
plt.plot(test_score, label='Loss on test set')
plt.legend()
plt.show()

Again, do a 5-fold cross validation at this point. How does the score compare to the earlier one?

In [ ]:
# compute cross validation score
scores = cross_val_score(gbr500, X, y, cv=5, scoring='neg_mean_absolute_error')
print('Cross validation score: {}'.format(-1 * np.mean(scores)))

##  5. Systematic hyperparameter tuning

Hand tuning a large number of hyperparameters is laborious. Luckily, `sklearn` provides a function [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to automate searches in the hyperparameter space. Even though, performing a grid-search of all of the hyperparameters at once would again lead to a combinatorial explosion. A general strategy for tuning hyperparameters in gradient boosted trees has been suggested [here](https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/). 

1. Choose a relatively high learning rate. Generally the default value of 0.1 works but somewhere between 0.05 to 0.2 should work for different problems.
2. Determine the optimum number of trees for this learning rate. This should range around 40 to 90. Remember to choose a value on which your system can work fairly fast. This is because it will be used for testing various scenarios and determining the tree parameters.
3. Tune tree-specific parameters for decided learning rate and number of trees. 
4. Lower the learning rate and increase the estimators proportionally to get more robust models.

We will follow the above process to tune our regressor.


### Step 1 & 2: Optimise `n_estimators` with `learning_rate=0.1`

In [ ]:
# candidates
param_test_n_est = {'n_estimators': range(40, 90, 10)}

# create the regressor
gbr_n_est = GradientBoostingRegressor(loss='lad', learning_rate=0.1, 
                                      max_features=len(cols), max_depth=10,
                                      min_samples_split=50, subsample=0.9,
                                      random_state=0)

# define hyperparameter search
gsearch = GridSearchCV(estimator= gbr_n_est, param_grid = param_test_n_est, 
                       scoring='neg_median_absolute_error', cv=5)

# perform search
gsearch.fit(X, y)

In [ ]:
# print best n_estimators
gsearch.best_params_

### Step 3: Optimise tree parameters with best `n_estimators`

Here we consider `max_depth` and `min_samples_split`:

In [ ]:
# candidates
param_test_tree = {'max_depth': range(5, 16, 2), 
                   'min_samples_split': range(10, 100, 20)}

# create the regressor
gbr_tree = GradientBoostingRegressor(loss='lad', learning_rate=0.1, 
                                     max_features=len(cols), subsample=0.9,
                                     n_estimators=70, random_state=0)

# define hyperparameter search
gsearch = GridSearchCV(estimator= gbr_tree, param_grid = param_test_tree, 
                       scoring='neg_median_absolute_error', cv=5)

# perform search
gsearch.fit(X, y)

In [ ]:
# print best max_depth and min_samples_split
gsearch.best_params_

### Step 4: Lower `learning_rate` and increase `n_estimators`

Here we use a factor of 5, so `learning_rate` is lowered to 0.02 and `n_estimators` increased to 350:

In [ ]:
# create the "optimised" regressor
gbr_opt = GradientBoostingRegressor(loss='lad', learning_rate=0.02, 
                                    max_features=len(cols), max_depth=7,
                                    min_samples_split=10, subsample=0.9,
                                    n_estimators=350, random_state=0)

# fit the model
gbr_opt.fit(X, y)

Eventually, we can use our "optimised" model to make predictions and compute CV scores:

In [ ]:
# make predictions
y_pred = gbr_opt.predict(X)

# plot the original and predicted data against each other
plt.figure(dpi=100)
plt.scatter(y, y_pred)
plt.show()

# compute cross validation score
scores = cross_val_score(gbr_opt, X, y, cv=5, scoring='neg_mean_absolute_error')
print('Cross validation score: {}'.format(-1 * np.mean(scores)))

**Yes, our efforts pay off**, as shown by the figure and the CV score!

---

## Exercises 

Similar to [classification_decision_tree.ipynb](classification_decision_tree.ipynb), use regression trees to fit one or some of the standard "toy" datasets embedded in `sklearn`, such as `boston-house-prices` and `diabetes`. These datasets are less challenging than our example.

In [ ]:
# load iris dataset
boston = sklearn.datasets.load_boston()
print(boston['DESCR'])